In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('cardio_train.csv', sep = ';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [7]:
df['years'] = (df['age'] / 365).round(0)
df['years'] = pd.to_numeric(df['years'], downcast = 'integer')
df['Cardio'] = df['cardio']

In [8]:
df = df.drop('age', axis = 1)

In [9]:
#df = df.drop('id', axis = 1)
df = df.drop('cardio', axis = 1)

In [10]:
X = np.array(df.iloc[:, :-1].values, dtype = np.float128)
Y = df.iloc[:, -1].values

In [11]:
df.head()

,id,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,years,Cardio
0,0,2,168,62.0,110,80,1,1,0,0,1,50,0
1,1,1,156,85.0,140,90,3,1,0,0,1,55,1
2,2,1,165,64.0,130,70,3,1,0,0,0,52,1
3,3,2,169,82.0,150,100,1,1,0,0,1,48,1
4,4,1,156,56.0,100,60,1,1,0,0,0,48,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 1)

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
model = LogisticRegression()

In [15]:
model.fit(X_train, Y_train)

LogisticRegression()

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, model.predict(X_test))

TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]

print(cm)
print((TP+TN)/(TP+TN+FP+FN))

[[6464 2145]
 [3120 5771]]
0.6991428571428572


In [17]:
model.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0])

In [18]:
Y_test

array([1, 1, 1, ..., 1, 1, 0])

98.31

In [19]:
lst = [2.       , 170.       ,  80.       , 130.       ,  90.       ,
         1.       ,   1.       ,   0.       ,   0.       ,   1.       ,
        27.6816609,  58.       ]

In [20]:
lst

[2.0, 170.0, 80.0, 130.0, 90.0, 1.0, 1.0, 0.0, 0.0, 1.0, 27.6816609, 58.0]

In [21]:
type(lst)

list

In [22]:
arr = np.array([lst])

In [23]:
arr

array([[  2.       , 170.       ,  80.       , 130.       ,  90.       ,
          1.       ,   1.       ,   0.       ,   0.       ,   1.       ,
         27.6816609,  58.       ]])

In [24]:
model.predict(arr)[0]

1

In [25]:
p =(model.predict_proba(arr))
round(p[0][1] * 100, 2)

98.31

In [26]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [39]:
solvers = ['lbfgs','newton-cg','liblinear','sag','saga']
penalty = ['l1', 'l2', 'elasticnet', 'none']

In [40]:
c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

In [41]:
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_test, Y_test)

In [42]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.716895 using {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}


In [37]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = GridSearchCV(LogisticRegression(penalty='l2'), param_grid)
grid_search = GridSearchCV(cv=None,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})
grid_result = grid_search.fit(X_test, Y_test)

In [38]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.694800 using {'C': 0.001}
